# Code to train model on recognizing different hand signs.

In [1]:
import pickle
import matplotlib.pyplot as plt # For data viz
import pandas as pd
import numpy as np
import sys
from tqdm.notebook import tqdm
import os
MAIN_DIR = os.path.dirname(os.path.abspath('__file__'))

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm

C:\Users\Phuong Hoang Dinh\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


# Dataset Generation

In [3]:
#load data
data_dict = pickle.load(open('data.p', 'rb'))

data = data_dict['data']
labels = data_dict['labels']

In [4]:
print(data[0])

[[0.06068733334541321, 0.3100363612174988], [0.1444000005722046, 0.2712266445159912], [0.20791888236999512, 0.17321628332138062], [0.2224220633506775, 0.07306307554244995], [0.23784911632537842, 0.0], [0.15568631887435913, 0.07333582639694214], [0.18263676762580872, 0.013934284448623657], [0.1692524254322052, 0.1072419285774231], [0.15313789248466492, 0.11891710758209229], [0.10369017720222473, 0.07029926776885986], [0.1317923665046692, 0.024378836154937744], [0.1266593039035797, 0.13630786538124084], [0.112984299659729, 0.12890994548797607], [0.05375194549560547, 0.08144316077232361], [0.07957541942596436, 0.031121015548706055], [0.08280402421951294, 0.1376902163028717], [0.0687447190284729, 0.1369052529335022], [0.0, 0.09611031413078308], [0.027103543281555176, 0.0680772066116333], [0.03880774974822998, 0.13879692554473877], [0.02626165747642517, 0.14640474319458008]]


class SignDataset(Dataset):
    def __init__(self, data_dir):
        self.data = ImageFolder(data_dir)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

    @property
    def classes(self):
        return self.data.classes


dataset = SignDataset(data_dir = os.path.join(MAIN_DIR, "data"))
len(dataset)

In [7]:
class SignDataset(Dataset):
    def __init__(self, data, labels):
        self.data = [torch.tensor(seq, dtype=torch.float32).flatten() for seq in data]
        self.labels = torch.tensor([int(label) for label in labels], dtype=torch.long)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

    @property
    def classes(self):
        return self.data.classes

In [8]:
dataset = SignDataset(data, labels)
print(len(dataset))

4706


In [9]:
landmarks, label = dataset[4000]
print(landmarks)
print(label)

tensor([0.1923, 0.6914, 0.2643, 0.6740, 0.3293, 0.6314, 0.3821, 0.6032, 0.4171,
        0.5628, 0.3040, 0.4112, 0.3715, 0.3840, 0.3938, 0.4640, 0.3918, 0.5358,
        0.2489, 0.3666, 0.2850, 0.2062, 0.3047, 0.1009, 0.3162, 0.0000, 0.1853,
        0.3766, 0.1696, 0.2220, 0.1578, 0.1208, 0.1535, 0.0219, 0.1268, 0.4355,
        0.0662, 0.3362, 0.0274, 0.2749, 0.0000, 0.2059])
tensor(5)


# DataLoader Generation

In [10]:
dataloader = DataLoader(dataset, batch_size = 32, shuffle = True)

In [11]:
for data, label in dataloader:
    break

In [12]:
print(data.shape)

torch.Size([32, 42])


# Neural Network Construction

In [16]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2*21, 60),
            nn.ReLU(),
            nn.Linear(60, 26),
            nn.LogSoftmax(dim = 1)
        )

    def forward(self, x):
        return self.net(x)

In [17]:
model = NeuralNetwork()
print(model)

NeuralNetwork(
  (net): Sequential(
    (0): Linear(in_features=42, out_features=60, bias=True)
    (1): ReLU()
    (2): Linear(in_features=60, out_features=26, bias=True)
    (3): LogSoftmax(dim=1)
  )
)


In [18]:
ex_output = model(data)
ex_output.shape # [batch_size, num_classes]

torch.Size([32, 26])

# Loss & Optimizer

In [19]:
#Loss
criterion = nn.CrossEntropyLoss()
#Optimizer
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [20]:
criterion(ex_output, label)

tensor(3.2434, grad_fn=<NllLossBackward0>)

In [ ]:
# Save model for model inference
f = open('model.p', 'wb')
pickle.dump([model], f)
f.close()